In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [5]:
hello = get_biggest_table("/Users/nguyensiry/Downloads/filling_slot/documents/Chuong V Yeu cau ky thuat (1) 17.33.25.pdf",15)

Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value
Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value
Cannot set gray non-stroke color because /'Pattern5' is an invalid float value
Cannot set gray non-stroke color because /'Pattern6' is an invalid float value
Cannot set gray non-stroke color because /'Pattern7' is an invalid float value
Cannot set gray non-stroke color because /'Pattern8' is an invalid float value
Cannot set gray non-stroke color because /'Pattern1'

{
  "metadata": {
    "source_file": "Chuong V Yeu cau ky thuat (1) 17.33.25",
    "page": 2,
    "table_order": 1,
    "total_rows": 109,
    "total_columns": 3
  },
  "headers": [
    "STT",
    "hang_hoa",
    "yeu_cau_ky_thuat"
  ],
  "data": [
    {
      "row_index": 1,
      "values": {
        "STT": "I",
        "hang_hoa": "Tải giả xả acquy",
        "yeu_cau_ky_thuat": ""
      }
    },
    {
      "row_index": 2,
      "values": {
        "STT": "1",
        "hang_hoa": "Yêu cầu chung",
        "yeu_cau_ky_thuat": "-  Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc \nxuất xứ, có chứng nhận chất lượng sản phẩm của nhà \nsản xuất. \n-  Thiết bị mới 100% chưa qua sử dụng \n-  Thiết bị phải được sản xuất từ năm 2021 trở lại đây \n-  Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản \nxuất, tối thiểu 12 tháng."
      }
    },
    {
      "row_index": 3,
      "values": {
        "STT": "2",
        "hang_hoa": "Thông số kỹ thuật",
        "yeu_cau_ky_thuat": ""
    

In [6]:
data = hello["data"]

In [7]:
hello

{'metadata': {'source_file': 'Chuong V Yeu cau ky thuat (1) 17.33.25',
  'page': 2,
  'table_order': 1,
  'total_rows': 109,
  'total_columns': 3},
 'headers': ['STT', 'hang_hoa', 'yeu_cau_ky_thuat'],
 'data': [{'row_index': 1,
   'values': {'STT': 'I',
    'hang_hoa': 'Tải giả xả acquy',
    'yeu_cau_ky_thuat': ''}},
  {'row_index': 2,
   'values': {'STT': '1',
    'hang_hoa': 'Yêu cầu chung',
    'yeu_cau_ky_thuat': '-  Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc \nxuất xứ, có chứng nhận chất lượng sản phẩm của nhà \nsản xuất. \n-  Thiết bị mới 100% chưa qua sử dụng \n-  Thiết bị phải được sản xuất từ năm 2021 trở lại đây \n-  Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản \nxuất, tối thiểu 12 tháng.'}},
  {'row_index': 3,
   'values': {'STT': '2',
    'hang_hoa': 'Thông số kỹ thuật',
    'yeu_cau_ky_thuat': ''}},
  {'row_index': 4,
   'values': {'STT': '',
    'hang_hoa': 'Điện trở tiếp địa',
    'yeu_cau_ky_thuat': '< 0.1 Ohm'}},
  {'row_index': 5,
   'values': 

In [8]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [9]:
converted_data

[{'ten_san_pham': 'Tải giả xả acquy',
  'cac_muc': [{'ten_hang_hoa': 'Yêu cầu chung',
    'thong_so_ky_thuat': {'37727': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
     '6B99D': 'Thiết bị mới 100% chưa qua sử dụng',
     'C2532': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
     'D99AE': 'Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.'}},
   {'ten_hang_hoa': 'Thông số kỹ thuật',
    'thong_so_ky_thuat': {'CA03E': ['Điện trở tiếp địa', '< 0.1 Ohm'],
     '32F10': ['Độ ồn', '< 75 dB khi hoạt động hết công suất.'],
     '5B83F': ['Điều kiện nhiệt độ',
      '-  Dải nhiệt độ hoạt động: 0 ÷ 40°C,  -  Dải nhiệt độ lưu kho: -25°C ÷70°C.'],
     'CDB1A': ['Điều kiện độ ẩm',
      '-  Độ ẩm tương đối từ 5 ÷ 95%, không đọng sương.'],
     'C0F33': ['Điện áp acquy cao nhất – tính theo giá trị tuyệt đối',
      '≥ 270 V'],
     'B47C0': ['Điện áp acquy nhỏ nhất', '≤ 18 V'],
    

In [10]:
context_prompts = []
for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                value_str = ' '.join(value)
            else:
                value_str = value
            query = {
                key: f"{ten_san_pham} {ten_hang_hoa} {value_str}"
            }
            context_prompts.append(query)



In [11]:
print(context_prompts[12:15])

[{'FDF6E': 'Tải giả xả acquy Thông số kỹ thuật Dòng xả lớn nhất với dải điện áp  220V  DC  ÷  240V  DC)  – tương ứng với 1 bộ ≥ 70 A'}, {'4AC15': 'Tải giả xả acquy Thông số kỹ thuật Các chế độ xả -  Gồm 4 chế độ: dòng không đổi, công suất không đổi, theo đặc tính dòng cho trước, điều chỉnh thủ công.'}, {'28860': 'Tải giả xả acquy Thông số kỹ thuật Bước công suất điều chỉnh xả tải 100 W'}]


In [ ]:
# Lưu với tên file mặc định
with open('context_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(context_prompts, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu {len(context_prompts)} context prompts vào file: context_prompts.json")

In [12]:
def prompt_create_query(context_prompt):
    prompt =  f"""
    Bạn sẽ nhận được một đoạn mô tả kỹ thuật ngắn gọn (context_prompt), thường là các mảnh thông tin kỹ thuật rời rạc. Nhiệm vụ của bạn là chuyển đổi đoạn mô tả đó thành một truy vấn (query) hoàn chỉnh bằng ngôn ngữ tự nhiên, với mục tiêu:

- Diễn đạt lại đoạn mô tả dưới dạng một câu hỏi hoặc câu truy vấn rõ ràng, dễ hiểu.
- Giữ nguyên đầy đủ tất cả các thông tin kỹ thuật có trong câu gốc (chế độ, dòng điện, điện áp, bước công suất,...).
- Không được loại bỏ hoặc làm mờ bất kỳ chi tiết kỹ thuật nào.
- Truy vấn cần phù hợp để tìm kiếm thông tin tương đồng về mặt ngữ nghĩa trong một hệ thống truy xuất dữ liệu kỹ thuật.

Dưới đây là các ví dụ:

**Input:** "Tải giả xả acquy Thông số kỹ thuật Dòng xả lớn nhất với dải điện áp 220V DC ÷ 240V DC – tương ứng với 1 bộ ≥ 70 A"

**Output:** "Dòng xả lớn nhất của tải giả xả acquy là bao nhiêu khi dải điện áp nằm trong khoảng 220V DC đến 240V DC, và tương ứng với 1 bộ thì có đạt dòng xả ≥ 70A không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Các chế độ xả - Gồm 4 chế độ: dòng không đổi, công suất không đổi, theo đặc tính dòng cho trước, điều chỉnh thủ công."

**Output:** "Tải giả xả acquy hỗ trợ những chế độ xả nào? Có phải gồm dòng không đổi, công suất không đổi, theo đặc tính dòng định trước, và điều chỉnh thủ công không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Bước công suất điều chỉnh xả tải 100 W"

**Output:** "Bước điều chỉnh công suất xả tải nhỏ nhất của tải giả xả acquy là bao nhiêu? Có phải là 100W không?"

---
Trả lời từng `context_prompt` theo cấu trúc như trên.


    DỮ LIỆU ĐẦU VÀO:
    ---
    {context_prompt}
    ---

    CHỈ CẦN TRẢ VỀ CÂU TRUY VẤN.
    """
    return prompt

In [13]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
test = context_prompts[:10]
test

[{'37727': 'Tải giả xả acquy Yêu cầu chung Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.'},
 {'6B99D': 'Tải giả xả acquy Yêu cầu chung Thiết bị mới 100% chưa qua sử dụng'},
 {'C2532': 'Tải giả xả acquy Yêu cầu chung Thiết bị phải được sản xuất từ năm 2021 trở lại đây'},
 {'D99AE': 'Tải giả xả acquy Yêu cầu chung Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.'},
 {'CA03E': 'Tải giả xả acquy Thông số kỹ thuật Điện trở tiếp địa < 0.1 Ohm'},
 {'32F10': 'Tải giả xả acquy Thông số kỹ thuật Độ ồn < 75 dB khi hoạt động hết công suất.'},
 {'5B83F': 'Tải giả xả acquy Thông số kỹ thuật Điều kiện nhiệt độ -  Dải nhiệt độ hoạt động: 0 ÷ 40°C,  -  Dải nhiệt độ lưu kho: -25°C ÷70°C.'},
 {'CDB1A': 'Tải giả xả acquy Thông số kỹ thuật Điều kiện độ ẩm -  Độ ẩm tương đối từ 5 ÷ 95%, không đọng sương.'},
 {'C0F33': 'Tải giả xả acquy Thông số kỹ thuật Điện áp acquy cao nhất – tính theo giá trị tuyệt đối 

In [15]:

queries = []
for item in test:
    for key, value in item.items():  # mỗi item là 1 dict
        prompt = prompt_create_query(value)
        response = client.responses.create(
            model="gpt-4o-mini",
            input=prompt,
            temperature=0
        )

        output_text = response.output_text.strip()
        queries.append(
            {
                "key": key,
                "value": value,
                "query": output_text
            }
        )


In [16]:
queries

[{'key': '37727',
  'value': 'Tải giả xả acquy Yêu cầu chung Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'query': '"Tải giả xả acquy yêu cầu các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ và có chứng nhận chất lượng sản phẩm của nhà sản xuất. Điều này có đúng không?"'},
 {'key': '6B99D',
  'value': 'Tải giả xả acquy Yêu cầu chung Thiết bị mới 100% chưa qua sử dụng',
  'query': '"Tải giả xả acquy có phải là thiết bị mới 100% và chưa qua sử dụng không?"'},
 {'key': 'C2532',
  'value': 'Tải giả xả acquy Yêu cầu chung Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
  'query': '"Tải giả xả acquy cần đáp ứng yêu cầu gì về năm sản xuất? Có phải thiết bị phải được sản xuất từ năm 2021 trở lại đây không?"'},
 {'key': 'D99AE',
  'value': 'Tải giả xả acquy Yêu cầu chung Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.',
  'query': 'Thời gian bảo hành của tải giả xả acq

In [ ]:
[
    {
        "key":key,
        "value": value,
        [
            "chunk_1": {
           "content_chunk" : content_chunk,
            "metadata": metadata
            }
        ]
    }
]

In [17]:
queries

[{'key': '37727',
  'value': 'Tải giả xả acquy Yêu cầu chung Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'query': '"Tải giả xả acquy yêu cầu các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ và có chứng nhận chất lượng sản phẩm của nhà sản xuất. Điều này có đúng không?"'},
 {'key': '6B99D',
  'value': 'Tải giả xả acquy Yêu cầu chung Thiết bị mới 100% chưa qua sử dụng',
  'query': '"Tải giả xả acquy có phải là thiết bị mới 100% và chưa qua sử dụng không?"'},
 {'key': 'C2532',
  'value': 'Tải giả xả acquy Yêu cầu chung Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
  'query': '"Tải giả xả acquy cần đáp ứng yêu cầu gì về năm sản xuất? Có phải thiết bị phải được sản xuất từ năm 2021 trở lại đây không?"'},
 {'key': 'D99AE',
  'value': 'Tải giả xả acquy Yêu cầu chung Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.',
  'query': 'Thời gian bảo hành của tải giả xả acq

In [ ]:
        
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name=collection_name,
    client=client,
    aclient=aclient,
)
filters = MetadataFilters(
    filters=[
        MetadataFilter(key="file_name", operator=FilterOperator.EQ, value=file_name),
        MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
    ],
    condition=FilterCondition.AND,
)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

retriever = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters)

query = query

# --- Thay đổi từ đây ---
query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever
)

# 2. Thực hiện truy vấn qua Query Engine
print("Bắt đầu truy vấn với Query Engine...")
response = query_engine.retrieve(query)
text_content = ""
for node in response:
    text_content += node.get_content() + "\n"

In [ ]:
def search_rag(url: str, api_key: str, query: str, collection_name: str = "thong_tin_san_pham", file_name: str = "NetSure_732_User_Manual"):
    # Cấu hình client Qdrant
    client = QdrantClient(
        url=url,
        api_key=api_key,
    )
    aclient = AsyncQdrantClient(
        url=url,
        api_key=api_key,
    )
    # Khởi tạo Vector Store
    vector_store = QdrantVectorStore(
        collection_name=collection_name,
        client=client,
        aclient=aclient,
    )
    filters = MetadataFilters(
        filters=[
            MetadataFilter(key="file_name", operator=FilterOperator.EQ, value=file_name),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

    retriever = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters)

    query = query

    # --- Thay đổi từ đây ---
    query_engine = RetrieverQueryEngine.from_args(
        retriever=retriever
    )

    # 2. Thực hiện truy vấn qua Query Engine
    print("Bắt đầu truy vấn với Query Engine...")
    response = query_engine.retrieve(query)
    text_content = ""
    for node in response:
        text_content += node.get_content() + "\n"
    return text_content

In [ ]:
results = []

for query in queries:
    for key, value in query.items():
        vector = search_rag(value)
        results.append({
            "key": key,
            "value": value,
            "text_content": text_content  
        })

In [ ]:
def create_query(requirement: str, text_content: str):
    template = (
        "Based on the following text, create a comprehensive summary for the entire document.\n"
        "Document:\n---\n{requirement}\n {text_content}---\nSummary:"
    )
    prompt_template = PromptTemplate(template)
    response = Settings.llm.predict(prompt_template, requirement=requirement, text_content=text_content)
    return response.strip()

In [ ]:
result1 = []
for item in results:
    value = item["value"]
    vector = item["vector"]
    answer = create_query(value, text_content)